### Introduction

BERTopic is a topic modeling technique (which is helpful to discover latent topics in collections of documents) that leverages pre-trained transformer based language model for topic embedding and can cluster these embeddings to generate topic representations with the classed-based TF-IDF procedure. With embeddings of lower dimension, it creates dense clusters, allowing for easily interpretable topics whilst keeping important words in the topic descriptions. It supports all kinds of modeling techniques: guided, supervised and semi-supervised, manual, multi-topic distributions, hierarchical, etc.

BERTopic can solve a problem with LDA model, that is, the losing of semantic relationships among words with BoG representation which ignores the order of them. In recent years, with the proposal of word embedding and transformer model, BERT and its variations have shown great results in generating contextual word and sentence vector representations, where semantically similar words or sentences are also closer.

The detailed information of this model is as follows: the authors first created document embeddings using a pre-trained language model (with Sentence-BERT to transfer sentences or paragraphs into dense vectors) to obtain document level information. Second, they reduced the dimensionality of the embeddings followed by the creation of semantically similar clusters of documents that represent specific topics. Here each cluster is basically a topic. Finally, they developed a class based TF-IDF to extract the topic representation from each topic to avoid the centroid based perspective.

(Grootendorst, 2022)

### Libraries

In [16]:
import numpy as np
import pandas as pd
import os

from bertopic import BERTopic
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

### Data and Preprocessing

In [3]:
data = np.load('literature.npy', allow_pickle=True).item()

In [5]:
# Extract papers from dictionary and save in a list
texts = []

for _, sections in data.items():
    full_text = " ".join(sections.values())
    texts.append(full_text)

### BERTopic

In [17]:
nltk.download("stopwords")
stop_words = stopwords.words('english')

vectorizer_model = CountVectorizer(stop_words=stop_words)

[nltk_data] Downloading package stopwords to C:\Users\Songlin
[nltk_data]     Wang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
topic_model = BERTopic(hdbscan_model=HDBSCAN(min_cluster_size=2, min_samples=1), vectorizer_model=vectorizer_model, top_n_words=200)
topic, probs = topic_model.fit_transform(texts)

2025-12-10 01:29:26,841 - BERTopic - WARNING: Note that extracting more than 100 words from a sparse can slow down computation quite a bit.


### Results

In [36]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5,-1_migration_environmental_hawaweer_al,"[migration, environmental, hawaweer, al, peopl...","[Research has demonstrated that, in a variety ..."
1,0,5,0_migration_household_environmental_natural,"[migration, household, environmental, natural,...",[Sub-Saharan Africa (SSA) has the highest grow...
2,1,4,1_drought_households_mobility_women,"[drought, households, mobility, women, assets,...",[Research on the demographic consequences of e...
3,2,4,2_migration_environmental_benin_nigeria,"[migration, environmental, benin, nigeria, nig...",[This paper examines the role of environmental...
4,3,4,3_migration_conditions_environmental_land,"[migration, conditions, environmental, land, r...",[This study analyses the factors that cause in...
5,4,4,4_countries_temperature_migration_poor,"[countries, temperature, migration, poor, emig...",[This paper describes the conceptual and pract...
6,5,2,5_health_maasai_climate_refugees,"[health, maasai, climate, refugees, mental, pa...","[Objectives By 2050, over 250 million people w..."
7,6,2,6_urbanization_rainfall_zambia_households,"[urbanization, rainfall, zambia, households, p...",[We investigate the role that climatic change ...


In [37]:
topic[0]

0

### Evaluation

BERTopic doesn't work well with small dataset! Different from LDA, it's a clustering based model. It can not generate 'distribution of topics', but several clusters of topics, even though these papers might be very different from each other. With small sample data, HDBSCAN can easily consider all inputs as noise (topic -1). It can only generate a few topics even with min_cluster_size=2, which can already lead to unstable results.

One possible way is to split each paper into chunks but I can't tell if it works better or not.

Similar to LDA, BERTopic can automize text classification part, however, it can not really assign externally defined categorization codes, especially when pre-defined class names are not present in the sample. Thus, we have to manually annotate the papers based on topics we get, top words of such topics, and the matching topics of each paper. But BERTopic can lower the workload as well, compared with doing fully manual annotation. Here we first take paper 0 for evaluation. Its topic is likely to be '0'.

In [38]:
topic_model.get_topic(0)

[('migration', np.float64(0.05081459617730974)),
 ('household', np.float64(0.03252707987193754)),
 ('environmental', np.float64(0.02867076425997377)),
 ('natural', np.float64(0.026577056545025565)),
 ('households', np.float64(0.02530264217684636)),
 ('scale', np.float64(0.022293776495376928)),
 ('temporary', np.float64(0.020790370222612972)),
 ('conflict', np.float64(0.020299255382389428)),
 ('international', np.float64(0.019498343749667506)),
 ('may', np.float64(0.018400652324568847)),
 ('also', np.float64(0.01787435910986213)),
 ('models', np.float64(0.017075898994088578)),
 ('ssa', np.float64(0.016514657456302248)),
 ('intentions', np.float64(0.01614622948830573)),
 ('canada', np.float64(0.01587256458172428)),
 ('transition', np.float64(0.015769158475685992)),
 ('model', np.float64(0.01574538834065875)),
 ('study', np.float64(0.01571061274791126)),
 ('climaterelated', np.float64(0.01560381374117454)),
 ('zone', np.float64(0.015525005179943879)),
 ('al', np.float64(0.0154776689895400

In [39]:
data = [[0, 1, 1, 1, \
        0, 1, 0, 0, 0, \
        1, 1, 1, 0, \
        1, 0, 1, 0, \
        0, 1, 0, \
        0, 0, 1, 1, 0, 1, \
        1, 0, 0, 0, 0, 1, 0, \
        1, 0, 0]]
result = pd.DataFrame(data, columns=['Qualitative method', 'Quantitative method', 'Socio-demo-economic data', 'Environmental data', \
                       'Individuals', 'Households', 'Subnational groups', 'National groups', 'International groups', \
                       'Urban', 'Rural', 'Time frame considered', 'Foresight', \
                       'Rainfall pattern / Variability', 'Temperature change', 'Food scarcity / Famine / Food security', 'Drought / Aridity / Desertification', \
                       'Floods', 'Erosion / Soil fertility / Land degradation / Deforestation / Salinisation', 'Self assessment / Perceived environment', \
                       'Labour migration', 'Marriage migration', 'Refugees', 'International migration', 'Cross-border migration', 'Internal migration', \
                       'Rural to urban', 'Rural to rural', 'Circular / Seasonal', 'Long distance', 'Short distance', 'Temporal', 'Permanent', \
                       'Age', 'Gender', 'Ethnicity / Religion']).astype(str)

In [40]:
manual_result = pd.read_excel('manual.xlsx').astype(str)

In [41]:
manual_result = manual_result.iloc[[3]].drop(columns=['ID', 'AUTHOR', 'TITLE']).reset_index(drop=True)
manual_result

,Qualitative method,Quantitative method,Socio-demo-economic data,Environmental data,Individuals,Households,Subnational groups,National groups,International groups,Urban,...,Rural to urban,Rural to rural,Circular / Seasonal,Long distance,Short distance,Temporal,Permanent,Age,Gender,Ethnicity / Religion
0,0,1,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [42]:
bool_result = (result.iloc[0] == manual_result.iloc[0])
bool_result.mean(axis=0)

np.float64(0.75)